In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
import re
import time

In [54]:
juyuso = pd.read_csv("juyuso.csv", encoding="utf-8")

In [3]:
juyuso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539 entries, 0 to 538
Data columns (total 19 columns):
번호         539 non-null int64
사업장명       539 non-null object
소재지전체주소    535 non-null object
도로명전체주소    522 non-null object
개업일자       539 non-null int64
영업상태명      539 non-null object
폐업일자       0 non-null float64
휴업시작일자     103 non-null float64
휴업종료일자     103 non-null float64
재개업일자      80 non-null float64
소재지면적      502 non-null float64
소재지우편번호    0 non-null float64
사업개시예정일    539 non-null int64
사업구분명      539 non-null object
인가상태명      0 non-null float64
전화번호       521 non-null object
위치정보(X)    530 non-null float64
위치정보(Y)    530 non-null float64
상세영업상태명    539 non-null object
dtypes: float64(9), int64(3), object(7)
memory usage: 80.1+ KB


In [55]:
juyuso = juyuso.drop(["폐업일자","휴업종료일자","번호","개업일자","영업상태명","폐업일자","휴업시작일자","휴업종료일자","재개업일자","소재지우편번호","사업개시예정일","사업구분명","인가상태명","전화번호","상세영업상태명"], axis=1)

In [56]:
juyuso.drop(["위치정보(X)","위치정보(Y)"],axis=1,inplace=True)

In [57]:
juyuso

,사업장명,소재지전체주소,도로명전체주소,소재지면적
0,광호주유소,서울특별시 서대문구 홍은동 431번지,서울특별시 서대문구 연희로 266 (홍은동),1577.00
1,(주)안국에너지 삼보주유소,서울특별시 서대문구 연희동 194-32번지,서울특별시 서대문구 연희로 78 (연희동),1074.40
2,(주)중앙에너비스동교지점,서울특별시 서대문구 창천동 512-3번지,서울특별시 서대문구 연희로 12 (창천동),567.60
3,연희주유소,서울특별시 서대문구 연희동 89-7번지,서울특별시 서대문구 연희로 129 (연희동),988.40
4,삼미상사(주) 서부주유소,서울특별시 서대문구 연희동 446-4번지,서울특별시 서대문구 성산로 255 (연희동),1260.00
5,홍제동주유소,서울특별시 서대문구 홍제동 22-3번지,서울특별시 서대문구 통일로 372 (홍제동),826.00
6,행촌 제2주유소,서울특별시 서대문구 대신동 16-7번지,서울특별시 서대문구 성산로 490 (대신동),759.30
7,(주)마봉 신우주유소,서울특별시 서대문구 홍은동 379-6번지,서울특별시 서대문구 가좌로 106 (홍은동),709.00
8,남정주유소,서울특별시 서대문구 남가좌동 329-34번지,서울특별시 서대문구 모래내로 243 (남가좌동),332.20
9,(주)창원현대주유소,서울특별시 서대문구 홍은동 171-5번지,서울특별시 서대문구 통일로 537 (홍은동),947.00


In [58]:
juyuso["시"] = juyuso["소재지전체주소"]

In [59]:
juyuso

,사업장명,소재지전체주소,도로명전체주소,소재지면적,시
0,광호주유소,서울특별시 서대문구 홍은동 431번지,서울특별시 서대문구 연희로 266 (홍은동),1577.00,서울특별시 서대문구 홍은동 431번지
1,(주)안국에너지 삼보주유소,서울특별시 서대문구 연희동 194-32번지,서울특별시 서대문구 연희로 78 (연희동),1074.40,서울특별시 서대문구 연희동 194-32번지
2,(주)중앙에너비스동교지점,서울특별시 서대문구 창천동 512-3번지,서울특별시 서대문구 연희로 12 (창천동),567.60,서울특별시 서대문구 창천동 512-3번지
3,연희주유소,서울특별시 서대문구 연희동 89-7번지,서울특별시 서대문구 연희로 129 (연희동),988.40,서울특별시 서대문구 연희동 89-7번지
4,삼미상사(주) 서부주유소,서울특별시 서대문구 연희동 446-4번지,서울특별시 서대문구 성산로 255 (연희동),1260.00,서울특별시 서대문구 연희동 446-4번지
5,홍제동주유소,서울특별시 서대문구 홍제동 22-3번지,서울특별시 서대문구 통일로 372 (홍제동),826.00,서울특별시 서대문구 홍제동 22-3번지
6,행촌 제2주유소,서울특별시 서대문구 대신동 16-7번지,서울특별시 서대문구 성산로 490 (대신동),759.30,서울특별시 서대문구 대신동 16-7번지
7,(주)마봉 신우주유소,서울특별시 서대문구 홍은동 379-6번지,서울특별시 서대문구 가좌로 106 (홍은동),709.00,서울특별시 서대문구 홍은동 379-6번지
8,남정주유소,서울특별시 서대문구 남가좌동 329-34번지,서울특별시 서대문구 모래내로 243 (남가좌동),332.20,서울특별시 서대문구 남가좌동 329-34번지
9,(주)창원현대주유소,서울특별시 서대문구 홍은동 171-5번지,서울특별시 서대문구 통일로 537 (홍은동),947.00,서울특별시 서대문구 홍은동 171-5번지


In [47]:
driver =webdriver.Chrome()
driver.get("https://map.google.com")

In [164]:
len(juyuso)-1

538

In [51]:
driver.find_element_by_css_selector("#gs_lc50 .tactile-searchbox-input").clear()

In [62]:
juyu_location = list()
# 정규식으로 처리
p=re.compile(r'(@+.+,)')
i=0

for i in range(len(juyuso)):
    # 주소 뽑는 매크로
    
    if juyuso.iloc[i][2]=="NaN":
        driver.find_element_by_css_selector("#gs_lc50 .tactile-searchbox-input").send_keys(str(juyuso.iloc[i][1]))
    else:
        driver.find_element_by_css_selector("#gs_lc50 .tactile-searchbox-input").send_keys(str(juyuso.iloc[i][2]))
    
    
    time.sleep(0.1)
    driver.find_element_by_class_name("searchbox-searchbutton").click()
    time.sleep(2.5)
    driver.find_element_by_css_selector("#gs_lc50 .tactile-searchbox-input").clear()
    time.sleep(0.1)
    url=driver.current_url
    time.sleep(0.1)

    #좌표 정리
    location = p.findall(url)
    location=location[0].replace("@", '')
    location=location.split(",")
    del(location[2])
    
    
    juyu_location.append([juyuso.iloc[i][0],juyuso.iloc[i][1],juyuso.iloc[i][2],location])

In [66]:
pd.DataFrame(juyu_location).to_csv("11.csv")

In [71]:
juyu_location_raw = juyu_location.copy()

In [72]:
juyu_location_raw_1=juyu_location

In [84]:
juyu_location[2][-1][1]

'126.9304364'

In [85]:
count = 0
for i in range(len(juyu_location)-1):
    if juyu_location[i][-1][1] == juyu_location[i+1][-1][1]:
        if juyu_location[i][-1][0] == juyu_location[i+1][-1][0]:
            count +=1

In [94]:
result=[]
for i in juyu_location:
    result.extend(i)

In [95]:
cc=result

In [103]:
juyu_location[0]

['광호주유소',
 '서울특별시 서대문구 홍은동 431번지 ',
 '서울특별시 서대문구 연희로 266 (홍은동)',
 ['37.5807121', '126.9343882']]

In [295]:
juyu_location = pd.DataFrame(juyu_location)

In [300]:
juyu_location.head(1)

,0,1
0,37.5806434,126.9342567


In [321]:
juyu_location=juyu_location.rename(columns={1:"Y"})
# df = df.rename(columns = {'Column Name': 'New Name'})

In [323]:
juyu_location.to_csv("juyuso_seoul.csv")

In [327]:
result=[]
for i in juyu_new:
    result.extend(i)

In [333]:
result[0]

['37.5806434', '126.9342567']

In [335]:
m= folium.Map([result[0][0],result[0][1]])

In [337]:
for i in range(len(result)-1):
    folium.Marker([result[i][0],result[i][1]]).add_to(m)

In [149]:
juyu_location_raw_1=juyu_location_raw.copy()

In [150]:
for i in range(len(juyu_location_raw_1)):
    juyu_location_raw_1[i].extend(juyu_location_raw_1[i][3])

In [154]:
ka=pd.DataFrame(juyu_location_raw_1)

In [157]:
ka.drop(columns=[3,6,7,8,9,10,11], inplace=True)

In [161]:
ka.head()

,0,1,2,4,5
0,광호주유소,서울특별시 서대문구 홍은동 431번지,서울특별시 서대문구 연희로 266 (홍은동),37.5807121,126.9343882
1,(주)안국에너지 삼보주유소,서울특별시 서대문구 연희동 194-32번지,서울특별시 서대문구 연희로 78 (연희동),37.5649687,126.9277413
2,(주)중앙에너비스동교지점,서울특별시 서대문구 창천동 512-3번지,서울특별시 서대문구 연희로 12 (창천동),37.5614034,126.9304364
3,연희주유소,서울특별시 서대문구 연희동 89-7번지,서울특별시 서대문구 연희로 129 (연희동),37.5692438,126.9303007
4,삼미상사(주) 서부주유소,서울특별시 서대문구 연희동 446-4번지,서울특별시 서대문구 성산로 255 (연희동),37.5677169,126.9167661


In [168]:
ka['동']=ka.iloc[:,1].str.split(" ", n = 3, expand = True).iloc[:,2]

In [172]:
ka.to_csv("forga.csv",encoding='cp949')

In [170]:
len(juyu_location_raw)

539

In [143]:
juyu_location=pd.read_csv("11.csv")

In [146]:
for i in range(len(juyu_location)):
    juyu_location[i].extend(juyu_location[i][3])

KeyError: 0